In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Imports
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.utils import resample
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, normalize
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import resample
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score
from scipy.stats import sem
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

# Performs one round of 5-fold cross validation
def tfcv(X, y, MODEL):
  y = np.array(y)
  acc, f1, pre, rec, auroc = [], [], [], [], []
  kf = StratifiedKFold(n_splits=5, shuffle=True)
  for train_index, test_index in kf.split(X, y):
    # Get the train/val data for the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # PCA was removed based off reveiwer's comment
    # pca = PCA(n_components=50)
    # X_train = pca.fit_transform(X_train)
    # X_test = pca.transform(X_test)

    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)

    MODEL.fit(X_train, y_train)
    y_pred = MODEL.predict(X_test)

    # Compute metrics for current fold
    fold_acc = accuracy_score(y_test, y_pred)
    fold_f1 = f1_score(y_test, y_pred)
    fold_pre = precision_score(y_test, y_pred)
    fold_rec = recall_score(y_test, y_pred)
    fold_roc = roc_auc_score(y_test, y_pred)
    acc.append(fold_acc)
    f1.append(fold_f1)
    pre.append(fold_pre)
    rec.append(fold_rec)
    auroc.append(fold_roc)

  return np.mean(np.array(acc)), np.mean(np.array(f1)), np.mean(np.array(pre)), np.mean(np.array(rec)), np.mean(np.array(auroc))

# Optimize a hyperparameter
def optimize(X, y, MODEL, PARAM, param_values):
    top_value = param_values[0]
    top_score = 0
    for i in tqdm(param_values):
        currentscore, _, _, _, _ = tfcv(X, y, MODEL(**{PARAM: i}, random_state=1))
        if currentscore > top_score:
            top_score = currentscore
            top_value = i

    final_acc, final_f1, final_pre, final_rec, final_auroc = [], [], [], [], []
    for i in range(5):
        ac, f1, pre, rec, auroc = tfcv(X, y, MODEL(**{PARAM: top_value}, random_state=1))
        final_acc.append(ac)
        final_f1.append(f1)
        final_pre.append(pre)
        final_rec.append(rec)
        final_auroc.append(auroc)
    final_acc = np.array(final_acc)
    final_f1 = np.array(final_f1)
    final_pre = np.array(final_pre)
    final_rec = np.array(final_rec)
    final_auroc = np.array(final_auroc)
    print(f'Accuracy: {np.mean(final_acc)}, +/- {sem(final_acc)}')
    print(f'F1: {np.mean(final_f1)}, +/- {sem(final_f1)}')
    print(f'Precision: {np.mean(final_pre)}, +/- {sem(final_pre)}')
    print(f'Recall: {np.mean(final_rec)}, +/- {sem(final_rec)}')
    print(f'Auroc: {np.mean(final_auroc)}, +/- {sem(final_auroc)}')
    print(f'top_value: {top_value}')
    return np.mean(final_acc), sem(final_acc)

# Load sequences from csv
df = pd.read_csv('../Data/LazBF_sequences.csv')
LazBF_sequences = df['sequences'].tolist()
LazBF_labels = np.array(df['labels'].tolist())

df = pd.read_csv('../Data/LazBF_sample.csv')
LazBF_sample = df['sequences'].tolist()
LazBF_sample_labels = np.array(df['labels'].tolist())

df = pd.read_csv('../Data/LazDEF_sequences.csv')
LazDEF_sequences = df['sequences'].tolist()
LazDEF_labels = np.array(df['labels'].tolist())

df = pd.read_csv('../Data/LazDEF_sample.csv')
LazDEF_sample = df['sequences'].tolist()
LazDEF_sample_labels = np.array(df['labels'].tolist())

# Load Embs
lazbf_mlm_none = np.load("../Embeddings/LazBF_mlm_none.npy")
lazdef_mlm_none = np.load("../Embeddings/LazDEF_mlm_none.npy")

lazbf_mlm_pa = np.load("../Embeddings/LazBF_mlm_PA.npy")
lazdef_mlm_pa = np.load("../Embeddings/LazDEF_mlm_PA.npy")

lazbf_mlm_lazbf = np.load("../Embeddings/LazBF_mlm_LazBF.npy")
lazdef_mlm_lazbf = np.load("../Embeddings/LazDEF_mlm_LazBF.npy")

lazbf_mlm_lazdef = np.load("../Embeddings/LazBF_mlm_LazDEF.npy")
# lazbf_mlm_lazdefLR04 = np.load("../Embeddings/LazBF_mlm_LazDEF_lr0.npy") # LR 3e-4 embeddings
# lazbf_mlm_lazdefLR05 = np.load("../Embeddings/LazBF_mlm_LazDEF_lr1.npy") # LR 3e-5 embeddings

lazdef_mlm_lazdef = np.load("../Embeddings/LazDEF_mlm_LazDEF.npy")
# lazdef_mlm_lazdefLR04 = np.load("../Embeddings/LazDEF_mlm_LazDEF_lr0.npy") # LR 3e-4 embeddings
# lazdef_mlm_lazdefLR05 = np.load("../Embeddings/LazDEF_mlm_LazDEF_lr1.npy") # LR 3e-5 embeddings

lazbf_mlm_lazbcdef = np.load("../Embeddings/LazBF_mlm_LazBCDEF.npy")
lazdef_mlm_lazbcdef = np.load("../Embeddings/LazDEF_mlm_LazBCDEF.npy")

# Hyperparameter grid
model_list = [LogisticRegression, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
param_list = ['C', 'n_estimators', 'n_estimators', 'C', 'hidden_layer_sizes']
value_list = [
   [1e-2, 1e-1, 1, 5], # LR
  #  [5, 25, 50, 80], # KNN was removed due to poor performance
   [5, 25, 50, 100], # RF
   [5, 25, 50, 100], # Ada
   [1e-2, 1e-1, 1, 5], # SVC
   [50, 100, 200, 500], # MLP
]

def balanced_sample_np(seqs, labels, N, seed):
    labels = np.array(labels)
    indices_0 = np.where(labels == 0)[0]
    indices_1 = np.where(labels == 1)[0]
    min_count = min(len(indices_0), len(indices_1), N // 2)
    sampled_indices_0 = np.random.choice(indices_0, min_count, replace=False)
    sampled_indices_1 = np.random.choice(indices_1, min_count, replace=False)
    sampled_indices = np.concatenate((sampled_indices_0, sampled_indices_1))
    np.random.seed(seed)
    random.seed(seed)
    np.random.shuffle(sampled_indices)
    return sampled_indices

In [19]:
#@title LazDEF LowN
np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 200, 42)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazdef prediction
  print(f"Low-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  3.99it/s]


Accuracy: 0.7389999999999999, +/- 0.004
F1: 0.7399777184318039, +/- 0.005525625059016988
Precision: 0.7436258564725383, +/- 0.005393753509868707
Recall: 0.746, +/- 0.010295630140986995
Auroc: 0.7390000000000001, +/- 0.00400000000000001
top_value: 0.01
Low-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  3.99it/s]


Accuracy: 0.7620000000000001, +/- 0.009565563234854496
F1: 0.7589466363821733, +/- 0.010120926688566956
Precision: 0.7688851047249218, +/- 0.011237464139382787
Recall: 0.758, +/- 0.011135528725660053
Auroc: 0.7619999999999999, +/- 0.009565563234854503
top_value: 0.1
Low-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.768, +/- 0.01044030650891052
F1: 0.7688714249689262, +/- 0.010865894264330716
Precision: 0.7688978554656034, +/- 0.013221810624342833
Recall: 0.776, +/- 0.013999999999999969
Auroc: 0.7680000000000001, +/- 0.010440306508910546
top_value: 100
Low-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


Accuracy: 0.932, +/- 0.001224744871391563
F1: 0.9330629084269685, +/- 0.0012169405888758443
Precision: 0.9261151319229123, +/- 0.0014899901603156864
Recall: 0.944, +/- 0.002449489742783171
Auroc: 0.932, +/- 0.001224744871391563
top_value: 50
Low-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:16<00:00,  4.02s/it]


Accuracy: 0.7609999999999999, +/- 0.005099019513592807
F1: 0.7564071663580874, +/- 0.006652824441700825
Precision: 0.7765156813991112, +/- 0.0011884981917575335
Recall: 0.744, +/- 0.012083045973594582
Auroc: 0.7609999999999999, +/- 0.005099019513592796
top_value: 100
[[0.7389999999999999, 0.7110000000000001, 0.717, 0.718, 0.743], [0.7620000000000001, 0.7110000000000001, 0.7229999999999999, 0.7060000000000001, 0.752], [0.804, 0.7740000000000001, 0.7230000000000001, 0.783, 0.768], [0.9490000000000001, 0.9440000000000002, 0.9260000000000002, 0.95, 0.932], [0.774, 0.71, 0.701, 0.743, 0.7609999999999999]]
[[0.004, 0.007648529270389171, 0.009165151389911676, 0.009823441352194252, 0.008000000000000004], [0.009565563234854496, 0.0053385391260156526, 0.016093476939431105, 0.010049875621120889, 0.005612486080160898], [0.0074833147735478955, 0.0053385391260156595, 0.007176350047203669, 0.005147815070493482, 0.01044030650891052], [0.0033166247903553808, 0.0018708286933869513, 0.004301162633521304,

In [6]:
#@title LazDEF MedN
np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 500, 42)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazdef prediction
  print(f"Med-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Med-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Med-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Med-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Med-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Med-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.33it/s]


Accuracy: 0.8228, +/- 0.0033823069050575674
F1: 0.8238244519536856, +/- 0.004224457694936874
Precision: 0.8201354701399846, +/- 0.0014735687910653488
Recall: 0.8304, +/- 0.008634813257969162
Auroc: 0.8228, +/- 0.00338230690505754
top_value: 0.1
Med-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.46it/s]


Accuracy: 0.82, +/- 0.0016733200530681391
F1: 0.8196772886984729, +/- 0.0015859515351754206
Precision: 0.8213516621812031, +/- 0.003977422784700294
Recall: 0.8224, +/- 0.005455272678794333
Auroc: 0.82, +/- 0.0016733200530681391
top_value: 0.1
Med-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.8535999999999999, +/- 0.003544009029333893
F1: 0.8561159297776193, +/- 0.00323554598900907
Precision: 0.8438812908489147, +/- 0.005144270100644205
Recall: 0.8712, +/- 0.0023323807579381183
Auroc: 0.8535999999999999, +/- 0.003544009029333836
top_value: 100
Med-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:20<00:00,  5.04s/it]


Accuracy: 0.9508000000000001, +/- 0.002059126028197403
F1: 0.9508332801218282, +/- 0.0019213427384850536
Precision: 0.9502893687815748, +/- 0.00393847569908117
Recall: 0.9527999999999999, +/- 0.0007999999999999784
Auroc: 0.9508000000000001, +/- 0.002059126028197394
top_value: 50
Med-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:40<00:00, 10.12s/it]


Accuracy: 0.82, +/- 0.004898979485566342
F1: 0.8219681984498457, +/- 0.005480876217996082
Precision: 0.8123568021273035, +/- 0.0034937695184307036
Recall: 0.8336, +/- 0.0077562877718661315
Auroc: 0.8200000000000001, +/- 0.00489897948556637
top_value: 50
[[0.8228, 0.7512000000000001, 0.7348, 0.7836000000000001, 0.7868], [0.82, 0.77, 0.7671999999999999, 0.7863999999999999, 0.7964], [0.8583999999999999, 0.8012, 0.7992000000000001, 0.8284, 0.8535999999999999], [0.9515999999999998, 0.9559999999999998, 0.954, 0.9560000000000001, 0.9508000000000001], [0.844, 0.7668000000000001, 0.7555999999999999, 0.8144, 0.82]]
[[0.0033823069050575674, 0.003382306905057547, 0.008064738061462374, 0.003600000000000003, 0.006651315659326361], [0.0016733200530681391, 0.004816637831516922, 0.007059745037889142, 0.0027856776554368514, 0.005564171097297435], [0.004874423042781561, 0.0026532998322843456, 0.006183849933496142, 0.0036000000000000042, 0.003544009029333893], [0.0023151673805580455, 0.0008944271909999167

In [7]:
#@title LazDEF HighN
np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 42)

y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazdef prediction
  print(f"High-N, LazDEF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_none[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"High-N, LazDEF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_pa[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"High-N, LazDEF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbf[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"High-N, LazDEF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"High-N, LazDEF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazdef_mlm_lazbcdef[idxs], LazDEF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

High-N, LazDEF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.42it/s]


Accuracy: 0.8674, +/- 0.0030099833886584673
F1: 0.8683821037038282, +/- 0.0031279488524124984
Precision: 0.8638400485664544, +/- 0.0027200696694857935
Recall: 0.874, +/- 0.003794733192202085
Auroc: 0.8674, +/- 0.0030099833886584673
top_value: 0.1
High-N, LazDEF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.04it/s]


Accuracy: 0.8630000000000001, +/- 0.0015491933384829716
F1: 0.8625889424806121, +/- 0.0016328261401224876
Precision: 0.8661081595956599, +/- 0.002239418998564068
Recall: 0.8603999999999999, +/- 0.003187475490101856
Auroc: 0.8630000000000001, +/- 0.0015491933384829716
top_value: 5
High-N, LazDEF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.8821999999999999, +/- 0.0035128336140500785
F1: 0.8841173586879236, +/- 0.0035384634801531125
Precision: 0.8698892979033378, +/- 0.0031098122535295624
Recall: 0.8996000000000001, +/- 0.005075431016179813
Auroc: 0.8821999999999999, +/- 0.0035128336140500655
top_value: 100
High-N, LazDEF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:35<00:00,  8.86s/it]


Accuracy: 0.9642, +/- 0.0010677078252031371
F1: 0.9641790036658147, +/- 0.0010439184046972404
Precision: 0.9644957351091954, +/- 0.001345391987038673
Recall: 0.9644, +/- 0.000748331477354771
Auroc: 0.9642, +/- 0.0010677078252031278
top_value: 50
High-N, LazDEF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [01:20<00:00, 20.22s/it]


Accuracy: 0.85, +/- 0.0017888543819998114
F1: 0.8508347946528699, +/- 0.0021987509652480622
Precision: 0.845059364858969, +/- 0.0017483973386930212
Recall: 0.8580000000000002, +/- 0.0051380930314660535
Auroc: 0.85, +/- 0.00178885438199983
top_value: 50
[[0.8674, 0.7807999999999999, 0.7767999999999999, 0.8156000000000001, 0.8124], [0.8630000000000001, 0.7936, 0.7806000000000001, 0.8154, 0.836], [0.8984, 0.8081999999999999, 0.8395999999999999, 0.8692, 0.8821999999999999], [0.9686, 0.9673999999999999, 0.9634, 0.9681999999999998, 0.9642], [0.8876000000000002, 0.7811999999999999, 0.7928000000000001, 0.8326, 0.85]]
[[0.0030099833886584673, 0.0036110940170535418, 0.006865857557508741, 0.0029257477676655416, 0.001536229149573728], [0.0015491933384829716, 0.004202380277890136, 0.003458323293158104, 0.002638181191654581, 0.0028106938645110296], [0.002135415650406234, 0.00293938769133976, 0.004707440918375944, 0.002059126028197393, 0.0035128336140500785], [0.0006782329983125471, 0.000509901951359

In [16]:
#@title LazBF LowN

np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 200, 42)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Low-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Low-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Low-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Low-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Low-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Low-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.62it/s]


Accuracy: 0.868, +/- 0.0025495097567963944
F1: 0.8670443279304969, +/- 0.003446128622598829
Precision: 0.8709614546324735, +/- 0.0034416746576243383
Recall: 0.868, +/- 0.005830951894845304
Auroc: 0.868, +/- 0.0025495097567963944
top_value: 0.1
Low-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.63it/s]


Accuracy: 0.877, +/- 0.005830951894845287
F1: 0.8785867750830583, +/- 0.005697179544604169
Precision: 0.8728210493295162, +/- 0.005779407601436075
Recall: 0.89, +/- 0.007071067811865473
Auroc: 0.877, +/- 0.005830951894845287
top_value: 0.01
Low-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.978, +/- 0.0012247448713916173
F1: 0.9783587331258466, +/- 0.0013920575120741714
Precision: 0.9721765480895916, +/- 0.00046242098424746376
Recall: 0.9860000000000001, +/- 0.0024494897427831527
Auroc: 0.978, +/- 0.0012247448713916173
top_value: 100
Low-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:09<00:00,  2.32s/it]


Accuracy: 0.8859999999999999, +/- 0.004301162633521326
F1: 0.8868662225051753, +/- 0.004462867216688891
Precision: 0.8863641465323386, +/- 0.004254557901996115
Recall: 0.8939999999999999, +/- 0.008124038404635967
Auroc: 0.8859999999999999, +/- 0.004301162633521326
top_value: 500
Low-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:12<00:00,  3.05s/it]


Accuracy: 0.865, +/- 0.004472135954999583
F1: 0.8639673769883043, +/- 0.0037988564765372556
Precision: 0.8752785713411638, +/- 0.008077162863184929
Recall: 0.8619999999999999, +/- 0.0037416573867739447
Auroc: 0.865, +/- 0.004472135954999558
top_value: 100
[[0.868, 0.86, 0.842, 0.8639999999999999, 0.8550000000000001], [0.877, 0.867, 0.8320000000000001, 0.8610000000000001, 0.8450000000000001], [0.9789999999999999, 0.977, 0.977, 0.9800000000000001, 0.978], [0.8880000000000001, 0.8880000000000001, 0.8619999999999999, 0.89, 0.8859999999999999], [0.8629999999999999, 0.8260000000000002, 0.8160000000000001, 0.8629999999999999, 0.865]]
[[0.0025495097567963944, 0.005700877125495719, 0.010074720839804983, 0.005787918451395127, 0.0038729833462073987], [0.005830951894845287, 0.00644204936336258, 0.010793516572461468, 0.012589678312014184, 0.006519202405202654], [0.0010000000000000009, 0.0012247448713916173, 0.00339116499156265, 0.0, 0.0012247448713916173], [0.0043588989435406605, 0.0081547532151500

In [17]:
#@title LazBF MedN

np.random.seed(1)
random.seed(1)
from sklearn.utils import check_random_state
random_state = check_random_state(1)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 500, 1)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"Med-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"Med-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"Med-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"Med-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"Med-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

Med-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.49it/s]


Accuracy: 0.884, +/- 0.0033466401061363346
F1: 0.8849959757565282, +/- 0.0032578484526496897
Precision: 0.8796273411379811, +/- 0.0030360490960744265
Recall: 0.8927999999999999, +/- 0.0040792156108742155
Auroc: 0.884, +/- 0.0033466401061363247
top_value: 0.1
Med-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:01<00:00,  2.74it/s]


Accuracy: 0.8907999999999999, +/- 0.00313687742827163
F1: 0.8922697865484401, +/- 0.0035990666951387445
Precision: 0.8819978518271281, +/- 0.0018636179793203123
Recall: 0.9048, +/- 0.006374950980203711
Auroc: 0.8907999999999999, +/- 0.00313687742827163
top_value: 0.01
Med-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.9564, +/- 0.0017204650534085254
F1: 0.9565348441945604, +/- 0.0017881323893264565
Precision: 0.9530534438761118, +/- 0.0013158536232715213
Recall: 0.9608000000000001, +/- 0.002332380757938145
Auroc: 0.9564, +/- 0.0017204650534085202
top_value: 50
Med-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:20<00:00,  5.17s/it]


Accuracy: 0.89, +/- 0.001549193338482968
F1: 0.8915861807690056, +/- 0.0013091236650251226
Precision: 0.8797544761937022, +/- 0.002715647043208811
Recall: 0.9056, +/- 0.003249615361854403
Auroc: 0.89, +/- 0.001549193338482968
top_value: 200
Med-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:32<00:00,  8.25s/it]


Accuracy: 0.8879999999999999, +/- 0.0014142135623731041
F1: 0.8898682366094087, +/- 0.0015469879169736978
Precision: 0.8768251141137651, +/- 0.0015809451750058356
Recall: 0.9047999999999998, +/- 0.0026532998322842923
Auroc: 0.8879999999999999, +/- 0.0014142135623731041
top_value: 500
[[0.884, 0.8535999999999999, 0.8619999999999999, 0.8752000000000001, 0.8756], [0.8907999999999999, 0.8564, 0.8628, 0.8868, 0.8968], [0.9551999999999999, 0.9655999999999999, 0.9576, 0.9648, 0.9564], [0.8832000000000001, 0.8824, 0.8695999999999999, 0.8964000000000001, 0.89], [0.8876000000000002, 0.8392, 0.8448, 0.8608, 0.8879999999999999]]
[[0.0033466401061363346, 0.0024819347291981904, 0.002756809750418063, 0.003136877428271616, 0.004019950248448349], [0.00313687742827163, 0.0023151673805580607, 0.005535341001239233, 0.004029888335922005, 0.003979949748426483], [0.0013564659966250401, 0.000748331477354789, 0.0016, 0.0004898979485566133, 0.0017204650534085254], [0.0036660605559646585, 0.0041665333311999364, 

In [18]:
#@title LazBF HighN

np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 42)
y_values = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]
y_errors = [
    [0, 0, 0, 0, 0], # vanilla-esm
    [0, 0, 0, 0, 0], # peptide-esm
    [0, 0, 0, 0, 0], # LazBF-esm
    [0, 0, 0, 0, 0], # LazDEF-esm
    [0, 0, 0, 0, 0], # LazBCDEF-esm
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"High-N, LazBF-task, Vanilla-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_none[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"High-N, LazBF-task, Peptide-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_pa[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"High-N, LazBF-task, LazBF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbf[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  print(f"High-N, LazBF-task, LazDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[3][i] = m
  y_errors[3][i] = e

  print(f"High-N, LazBF-task, LazBCDEF-ESM-Embeddings {model}")
  m, e = optimize(lazbf_mlm_lazbcdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[4][i] = m
  y_errors[4][i] = e

  i += 1
print(y_values)
print(y_errors)

High-N, LazBF-task, Vanilla-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:04<00:00,  1.23s/it]


Accuracy: 0.9122, +/- 0.002853068523537426
F1: 0.9123142770697037, +/- 0.0030689397755547985
Precision: 0.9098615319591129, +/- 0.0014054768242513641
Recall: 0.9155999999999999, +/- 0.005564171097297396
Auroc: 0.9122, +/- 0.00285306852353744
top_value: 0.1
High-N, LazBF-task, Peptide-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.75it/s]


Accuracy: 0.9064, +/- 0.0019899748742132667
F1: 0.9066489715071766, +/- 0.0018201095750795325
Precision: 0.9061593685051095, +/- 0.003440587782922836
Recall: 0.908, +/- 0.0016733200530681658
Auroc: 0.9064, +/- 0.0019899748742132667
top_value: 1
High-N, LazBF-task, LazBF-ESM-Embeddings <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.9663999999999999, +/- 0.0010295630140987257
F1: 0.9660908517066185, +/- 0.0010402420988185354
Precision: 0.9743469625385739, +/- 0.0010189198441896367
Recall: 0.9583999999999999, +/- 0.0015999999999999944
Auroc: 0.9663999999999999, +/- 0.0010295630140987062
top_value: 50
High-N, LazBF-task, LazDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:36<00:00,  9.24s/it]


Accuracy: 0.9168000000000001, +/- 0.0024979991993593683
F1: 0.9172148394930002, +/- 0.002539230169658574
Precision: 0.9134149008242508, +/- 0.0023217390870711244
Recall: 0.9212, +/- 0.003382306905057566
Auroc: 0.9168000000000001, +/- 0.0024979991993593644
top_value: 100
High-N, LazBF-task, LazBCDEF-ESM-Embeddings <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [01:02<00:00, 15.62s/it]


Accuracy: 0.9038, +/- 0.0010198039027185535
F1: 0.9041152231421051, +/- 0.0010469190100800125
Precision: 0.9020250096067233, +/- 0.0022340621568664423
Recall: 0.9067999999999999, +/- 0.0030724582991474587
Auroc: 0.9038, +/- 0.0010198039027185513
top_value: 50
[[0.9122, 0.8734, 0.8699999999999999, 0.893, 0.9048], [0.9064, 0.8797999999999998, 0.875, 0.9011999999999999, 0.9072000000000001], [0.9654, 0.9704, 0.9676, 0.9709999999999999, 0.9663999999999999], [0.9074, 0.8912000000000001, 0.8866000000000002, 0.9182, 0.9168000000000001], [0.9044000000000001, 0.8468, 0.8736, 0.8984, 0.9038]]
[[0.002853068523537426, 0.0013266499161422004, 0.0036055512754640186, 0.002345207879911743, 0.0025573423705088838], [0.0019899748742132667, 0.0023108440016582576, 0.004110960958218865, 0.0012409673645990794, 0.0013564659966250564], [0.002111871208194274, 0.0011224972160321864, 0.0005099019513592789, 0.0012649110640673528, 0.0010295630140987257], [0.001208304597359473, 0.0019595917942265683, 0.002420743687382

---

In [13]:
#@title Comparing differnt learning rates on LazDEF task

np.random.seed(42)
random.seed(42)
from sklearn.utils import check_random_state
random_state = check_random_state(42)

idxs = balanced_sample_np(lazbf_mlm_none, LazBF_sample_labels, 1000, 42)
y_values = [
    [0, 0, 0, 0, 0], # lr=1e-4
    [0, 0, 0, 0, 0], # lr=1e-5
    [0, 0, 0, 0, 0], # lr=1e-6
]
y_errors = [
    [0, 0, 0, 0, 0], # lr=1e-4
    [0, 0, 0, 0, 0], # lr=1e-5
    [0, 0, 0, 0, 0], # lr=1e-6
]

i = 0
for model, param, grid in zip(model_list, param_list, value_list):

  # Lazbf prediction
  print(f"High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-4 {model}")
  m, e = optimize(lazbf_mlm_lazdefLR04[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[0][i] = m
  y_errors[0][i] = e

  print(f"High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-5 {model}")
  m, e = optimize(lazbf_mlm_lazdefLR05[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[1][i] = m
  y_errors[1][i] = e

  print(f"High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-6 {model}")
  m, e = optimize(lazbf_mlm_lazdef[idxs], LazBF_sample_labels[idxs], model, param, grid)
  y_values[2][i] = m
  y_errors[2][i] = e

  i += 1
print(y_values)
print(y_errors)

High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-4 <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


Accuracy: 0.9065999999999999, +/- 0.0029765752132274704
F1: 0.9068797030368939, +/- 0.0031288426420821615
Precision: 0.9019473057258909, +/- 0.0032247339505695146
Recall: 0.9128000000000001, +/- 0.003878143885933043
Auroc: 0.9065999999999999, +/- 0.0029765752132274543
top_value: 5
High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-5 <class 'sklearn.linear_model._logistic.LogisticRegression'>


100%|██████████| 4/4 [00:02<00:00,  1.67it/s]


Accuracy: 0.9109999999999999, +/- 0.0014142135623730885
F1: 0.9112659027815552, +/- 0.0014564150811609447
Precision: 0.9106168058372741, +/- 0.0019769143294044098
Recall: 0.9132, +/- 0.0026532998322843287
Auroc: 0.9110000000000001, +/- 0.0014142135623730961
top_value: 1
High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-6 <class 'sklearn.linear_model._logistic.LogisticRegression'>


...

Accuracy: 0.8974, +/- 0.001939071942966517
F1: 0.8982368600109268, +/- 0.0019017257632056685
Precision: 0.8916082645181372, +/- 0.0028671193010212487
Recall: 0.9056000000000001, +/- 0.002785677655436822
Auroc: 0.8973999999999999, +/- 0.0019390719429665082
top_value: 100
High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-5 <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:52<00:00, 13.09s/it]


Accuracy: 0.9078000000000002, +/- 0.000800000000000016
F1: 0.9085550254932281, +/- 0.00075957322345353
Precision: 0.9017223779339242, +/- 0.0014887848094103288
Recall: 0.9164, +/- 0.0013266499161421208
Auroc: 0.9078000000000002, +/- 0.000800000000000016
top_value: 100
High-N, LazBF-task, LazDEF-ESM-Embeddings LR=1e-6 <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>


100%|██████████| 4/4 [00:41<00:00, 10.35s/it]


Accuracy: 0.914, +/- 0.0031144823004794846
F1: 0.9145204274222815, +/- 0.0029779064149716957
Precision: 0.9096999381778458, +/- 0.005356944345437027
Recall: 0.9200000000000002, +/- 0.0016733200530681528
Auroc: 0.914, +/- 0.003114482300479499
top_value: 50
[[0.9065999999999999, 0.844, 0.8558, 0.8872, 0.8974], [0.9109999999999999, 0.8710000000000001, 0.8802, 0.8974000000000002, 0.9078000000000002], [0.9092, 0.8942, 0.8888000000000001, 0.9164, 0.914]]
[[0.0029765752132274704, 0.003209361307176273, 0.0023323807579380936, 0.0011575836902790245, 0.001939071942966517], [0.0014142135623730885, 0.0014832396974191527, 0.004091454509095786, 0.0020149441679610017, 0.000800000000000016], [0.001319090595827293, 0.0017720045146669308, 0.002782085548648707, 0.0038026306683663306, 0.0031144823004794846]]


In [25]:
#@title Zero-shot predictions with downstream models

# Lown N - LazBF -> LazDEF
def zero_shot_for_downstream_model(Xtr, ytr, X_test, y_test, train_size, MODEL):
  idxs = balanced_sample_np(Xtr, ytr, train_size, 1)
  X_train = Xtr[idxs]
  y_train = ytr[idxs]
  scaler=StandardScaler()
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.transform(X_test)

  MODEL.fit(X_train, y_train)
  y_pred = MODEL.predict(X_test)
  print(accuracy_score(y_test, y_pred))

zero_shot_for_downstream_model(lazbf_mlm_none, LazBF_sample_labels, lazdef_mlm_none, LazDEF_sample_labels, 200, SVC(C=0.1))
zero_shot_for_downstream_model(lazbf_mlm_none, LazBF_sample_labels, lazdef_mlm_none, LazDEF_sample_labels, 500, RandomForestClassifier(n_estimators=50))
zero_shot_for_downstream_model(lazbf_mlm_none, LazBF_sample_labels, lazdef_mlm_none, LazDEF_sample_labels, 1000, SVC(C=1))

zero_shot_for_downstream_model(lazdef_mlm_none, LazDEF_sample_labels, lazbf_mlm_none, LazBF_sample_labels, 200, SVC(C=0.1))
zero_shot_for_downstream_model(lazdef_mlm_none, LazDEF_sample_labels, lazbf_mlm_none, LazBF_sample_labels, 500, SVC(C=0.1))
zero_shot_for_downstream_model(lazdef_mlm_none, LazDEF_sample_labels, lazbf_mlm_none, LazBF_sample_labels, 1000, LogisticRegression(C=1))

0.54218
0.58338
0.54668
0.70158
0.7214
0.70548


In [ ]:
!pip install transformers[torch] evaluate datasets
!pip install -U kaleido
!pip install captum